# make a box of eddies

In [ ]:
!pip install ecco_v4_py

In [ ]:
import numpy as np
import xarray as xr
import xmitgcm.llcreader as llcreader
%matplotlib inline
import matplotlib.pyplot as plt
import ecco_v4_py as ecco
import cmocean

#3D plot
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (10,10)

# Read in ECCO data

In [ ]:
## read in 3D data and get Theta
model = llcreader.ECCOPortalLLC2160Model()
ds_sst = model.get_dataset(varnames=['Theta'], k_levels=[0], type='latlon')
ds_sst

## regrid data onto new uniform grid using eccopy

In [ ]:
ecco_ds = ds_sst
new_grid_delta_lat = 360/8640
new_grid_delta_lon = 360/8640

new_grid_min_lat = -90+new_grid_delta_lat/2
new_grid_max_lat = 90-new_grid_delta_lat/2

new_grid_min_lon = -180+new_grid_delta_lon/2
new_grid_max_lon = 180-new_grid_delta_lon/2

new_grid_lon, new_grid_lat, field_nearest_1deg =\
        ecco.resample_to_latlon(ecco_ds.XC, \
                                ecco_ds.YC, \
                                ecco_ds.Theta.isel(time=6200),\
                                new_grid_min_lat, new_grid_max_lat, new_grid_delta_lat,\
                                new_grid_min_lon, new_grid_max_lon, new_grid_delta_lon,\
                                fill_value = np.NaN, \
                                mapping_method = 'nearest_neighbor',
                                radius_of_influence = 120000)

In [ ]:
da = xr.DataArray(field_nearest_1deg,name='Theta',coords={'lat':new_grid_lat[:,0],'lon':new_grid_lon[0,:]},dims=('lat','lon'))
da.coords['lon'] = np.mod(da['lon'], 360)
da = da.sortby(da.lon)


In [ ]:
plt.rcParams['figure.figsize'] = (10,5)
subset = da.sel(lat=slice(48,60),lon=slice(180,240))
subset.plot(vmin=3,vmax=10,cmap='cmo.thermal')

In [ ]:
111*(subset.lat[0]-subset.lat[1])

In [ ]:
subset25 = subset.coarsen(dim={'lat':4,'lon':4}, boundary="trim").mean()
print(111*(subset25.lat[0]-subset25.lat[1]))
subset50 = subset.coarsen(dim={'lat':12,'lon':12}, boundary="trim").mean()
print(111*(subset50.lat[0]-subset50.lat[1]))
da25 = da.coarsen(dim={'lat':4,'lon':4}, boundary="trim").mean()
print(111*(subset25.lat[0]-subset25.lat[1]))
da50 = da.coarsen(dim={'lat':12,'lon':12}, boundary="trim").mean()
                            

In [ ]:
import cartopy
import cartopy.crs as ccrs
ortho = ccrs.Orthographic(-170, 20)           # define target coordinate frame
geo = ccrs.PlateCarree()                     # define origin coordinate frame

plt.figure(figsize=(10,5))                    #set the figure size
ax = plt.subplot(1, 1, 1, projection=ortho)  #create the axis for plotting

q = da25.plot(ax=ax, 
                            transform = geo, 
                            cmap='cmo.thermal', 
                            vmin=4, 
                            vmax=12,cbar_kwargs={'label':'SST ($\circ$C)'}) # plot a colormap in transformed coordinates cmap='OrRd', 

global_extent = ax.get_extent(crs=ccrs.PlateCarree())
gg = global_extent[:2] + (45,57)
gg = (195,242)+gg[2:] 
ax.set_extent(gg, crs=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.LAND,color='gray')
plt.savefig('./../figures/GOA_25kmSST.png')
ax.add_feature(cartopy.feature.COASTLINE)

In [ ]:
import cartopy
import cartopy.crs as ccrs
ortho = ccrs.Orthographic(-170, 20)           # define target coordinate frame
geo = ccrs.PlateCarree()                     # define origin coordinate frame

plt.figure(figsize=(10,5))                    #set the figure size
ax = plt.subplot(1, 1, 1, projection=ortho)  #create the axis for plotting

q = da50.plot(ax=ax, 
                            transform = geo, 
                            cmap='cmo.thermal', 
                            vmin=4, 
                            vmax=12,cbar_kwargs={'label':'SST ($\circ$C)'}) # plot a colormap in transformed coordinates cmap='OrRd', 

global_extent = ax.get_extent(crs=ccrs.PlateCarree())
gg = global_extent[:2] + (45,57)
gg = (195,242)+gg[2:] 
ax.set_extent(gg, crs=ccrs.PlateCarree())
ax.add_feature(cartopy.feature.LAND,color='gray')
plt.savefig('./../figures/GOA_50kmSST.png')
ax.add_feature(cartopy.feature.COASTLINE)

In [ ]:
len2 = 111*np.abs(da.lat[0]-lat[1])
print(len2)
den_grad =  np.abs(np.gradient(da))#/len2)
result,xx = np.histogram(den_grad,bins=xx_in)


In [ ]:
gradxy =  np.abs(np.gradient(da))/4
grad=np.sqrt(gradxy[0,:,:]**2+gradxy[0,:,:]**2)

In [ ]:
plt.imshow(grad,vmin=0,vmax=.01)

In [ ]:
rlen.data

In [ ]:
length_scale = np.arange(1,50)
## create the empty data arrays to store the normalized histograms (normalized the *100 for percentage count)
xx_in = np.arange(0,.2,.001)
xx_in2 = np.arange(0,.2-.001,.001)
data = np.zeros((len(length_scale),len(xx_in2)))
ddn=xr.DataArray(data,dims=('length_scale','gradx'),coords={'length_scale':length_scale,'gradx':xx_in2})
for ilen2,len2 in enumerate(length_scale):
    da2 = da.coarsen(dim={'lat':len2,'lon':len2}, boundary="trim").mean()
    rlen = 111*np.abs(da2.lat[0]-da2.lat[1])
    gradxy =  np.abs(np.gradient(da2))/rlen.data
    grad=np.sqrt(gradxy[0,:,:]**2+gradxy[0,:,:]**2)
    result,xx = np.histogram(grad,bins=xx_in)
    ddn[ilen2,:]=ddn[ilen2,:]+result


In [ ]:
plt.imshow(ddn,vmin=0,vmax=100)

# explore the data and image surface

In [ ]:
#some nice eddies off of BC
ds_sst.Theta.isel(time=-1,j=slice(5450,5550),i=slice(6300,6400)).plot(vmin=6,vmax=8,cmap='turbo')

# make a subset and load the data
ecco data has a lot of dims/coords here I'm going to simplify by just putting it into a new data array to do this correctly use ecco_py, but I'm just doing a small region so okay to do this way


In [ ]:
subset = ds_sst.isel(time=-1,j=slice(5450,5550),i=slice(6300,6400),k=slice(0,40))
array = xr.DataArray(subset.Theta.data,dims=['depth','lat','lon'],coords={'depth':subset.Z.data,'lat':subset.YC[:,0].data,'lon':subset.XC[0,:].data})
#load the data to avoid streaming errors that sometimes pop up, also makes playing around with plots faster
array = array.load()

# make a box of eddies

In [ ]:
z_cut = array.isel(depth=0).data
x_cut = array.isel(lon=-1).data 
y_cut = array.isel(lat=0).data 
xx = array.lon
yy = array.lat
zz = array.depth

z_cut.shape

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
cmap,vmin,vmax='turbo',6.3,8.5

#offset is used to shift it to the top of the cube
X,Y=np.meshgrid(xx,yy)
ax.contourf(X,Y,z_cut,60,zdir='z',offset=zz[0],cmap=cmap,vmin=vmin,vmax=vmax)
Y,Z=np.meshgrid(yy,zz)
ax.contourf(x_cut,Y,Z,60,zdir='x',offset=xx[-1],cmap=cmap,vmin=vmin,vmax=vmax) 
X,Z=np.meshgrid(xx,zz)
ax.contourf(X,y_cut,Z,60,zdir='y',offset=yy[0],cmap=cmap,vmin=vmin,vmax=vmax)

#ax.set_frame_on(True)
ax.set_zlim(zz[-1],zz[0])
ax.set_xlim(xx[0],xx[-1])
ax.set_ylim(yy[0],yy[-1])
#ax.plot([xx[0],xx[-1]],[yy[0],yy[0]],zs=zz[0],zdir='z',color='k') #tried to add edge to contourf plots but shows up underneath

plt.savefig('./figures/3D_ecco_axes.png')
ax._axis3don = False
plt.savefig('./figures/3D_ecco_noaxes.png')

# flip it, reverse it, Ti esrever dna ti pilf, nwod gniht ym tup
look at other side of box

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (10,10)

array2 = array[:,:-10,:-10] #reduce because nan (land) in bottom of data

z_cut = array2.isel(depth=0).data
z_cut = np.flip(z_cut, axis=0) #flip the data, I want to see other side of eddy
x_cut = array2.isel(lon=-1).data 
x_cut = np.flip(x_cut, axis=1) #flip the data, I want to see other side of eddy
y_cut = array2.isel(lat=-1).data 
xx = array2.lon
yy = array2.lat
zz = array2.depth

#set up figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
cmap,vmin,vmax='turbo',6.3,8.5

#offset is used to shift it to the top of the cube
X,Y=np.meshgrid(xx,yy)
ax.contourf(X,Y,z_cut.data,60,zdir='z',offset=zz[0],cmap=cmap,vmin=vmin,vmax=vmax)
Y,Z=np.meshgrid(yy,zz)
ax.contourf(x_cut.data,Y,Z,60,zdir='x',offset=xx[-1],cmap=cmap,vmin=vmin,vmax=vmax) 
X,Z=np.meshgrid(xx,zz)
ax.contourf(X,y_cut.data,Z,60,zdir='y',offset=yy[0],cmap=cmap,vmin=vmin,vmax=vmax)
ax.set_zlim(zz[-1],zz[0])
ax.set_xlim(xx[0],xx[-1])
ax.set_ylim(yy[0],yy[-1])
plt.savefig('./figures/3D_ecco_axesR.png')
ax._axis3don = False
plt.savefig('./figures/3D_ecco_noaxesR.png')

# change colormap

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (10,10)

array2 = array[:,:-10,:-10] #reduce because nan (land) in bottom of data

z_cut = array2.isel(depth=0).data
z_cut = np.flip(z_cut, axis=0) #flip the data, I want to see other side of eddy
x_cut = array2.isel(lon=-1).data 
x_cut = np.flip(x_cut, axis=1) #flip the data, I want to see other side of eddy
y_cut = array2.isel(lat=-1).data 
xx = array2.lon
yy = array2.lat
zz = array2.depth

#set up figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
cmap,vmin,vmax='RdYlBu_r',6.3,8.5

#offset is used to shift it to the top of the cube
X,Y=np.meshgrid(xx,yy)
ax.contourf(X,Y,z_cut.data,60,zdir='z',offset=zz[0],cmap=cmap,vmin=vmin,vmax=vmax)
Y,Z=np.meshgrid(yy,zz)
ax.contourf(x_cut.data,Y,Z,60,zdir='x',offset=xx[-1],cmap=cmap,vmin=vmin,vmax=vmax) 
X,Z=np.meshgrid(xx,zz)
ax.contourf(X,y_cut.data,Z,60,zdir='y',offset=yy[0],cmap=cmap,vmin=vmin,vmax=vmax)
ax.set_zlim(zz[-1],zz[0])
ax.set_xlim(xx[0],xx[-1])
ax.set_ylim(yy[0],yy[-1])
plt.savefig('./figures/3D_ecco_axesRa.png')
ax._axis3don = False
plt.savefig('./figures/3D_ecco_noaxesRa.png')

# bigger box of eddies

In [ ]:
subset = ds_sst.isel(time=-1,j=slice(5350,5550),i=slice(6200,6400),k=slice(0,40))
array = xr.DataArray(subset.Theta.data,dims=['depth','lat','lon'],coords={'depth':subset.Z.data,'lat':subset.YC[:,0].data,'lon':subset.XC[0,:].data})
#load the data to avoid streaming errors that sometimes pop up, also makes playing around with plots faster
array = array.load()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (20,20)

array2 = array[:,:-10,:-10] #reduce because nan (land) in bottom of data

z_cut = array2.isel(depth=0).data
z_cut = np.flip(z_cut, axis=0) #flip the data, I want to see other side of eddy
x_cut = array2.isel(lon=-1).data 
x_cut = np.flip(x_cut, axis=1) #flip the data, I want to see other side of eddy
y_cut = array2.isel(lat=-1).data 
xx = array2.lon
yy = array2.lat
zz = array2.depth

#set up figure
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
cmap,vmin,vmax='RdYlBu_r',5.3,8.5

#offset is used to shift it to the top of the cube
X,Y=np.meshgrid(xx,yy)
ax.contourf(X,Y,z_cut.data,60,zdir='z',offset=zz[0],cmap=cmap,vmin=vmin,vmax=vmax)
Y,Z=np.meshgrid(yy,zz)
ax.contourf(x_cut.data,Y,Z,60,zdir='x',offset=xx[-1],cmap=cmap,vmin=vmin,vmax=vmax) 
X,Z=np.meshgrid(xx,zz)
ax.contourf(X,y_cut.data,Z,60,zdir='y',offset=yy[0],cmap=cmap,vmin=vmin,vmax=vmax)
ax.set_zlim(zz[-1],zz[0])
ax.set_xlim(xx[0],xx[-1])
ax.set_ylim(yy[0],yy[-1])


color = '0.5' #color of the line of the corners
#Get xlim,ylim and zlim
xlim,ylim,zlim = list(map(np.array,[ax.get_xlim(),ax.get_ylim(),ax.get_zlim()]))
#Plot corners
ax.plot(xlim*0+xlim[1],ylim,zlim*0,color,linewidth=1,zorder=1e4)
ax.plot(xlim*0+xlim[1],ylim,zlim*0+zlim[-2],color,linewidth=2,zorder=1e4)
ax.plot(xlim*0+xlim[-2],ylim,zlim*0,color,linewidth=1,zorder=1e4)
ax.plot(xlim,ylim*0+ylim[0],zlim*0,color,linewidth=1,zorder=1e4)
ax.plot(xlim,ylim*0+ylim[0],zlim*0+zlim[-2],color,linewidth=2,zorder=1e4)
ax.plot(xlim,ylim*0+ylim[-1],zlim*0,color,linewidth=1,zorder=1e4)
ax.plot(xlim*0+xlim[1],ylim*0+ylim[0],zlim,color,linewidth=1,zorder=1e4)
ax.plot(xlim*0+xlim[-2],ylim*0+ylim[0],zlim,color,linewidth=2,zorder=1e4)
ax.plot(xlim*0+xlim[1],ylim*0+ylim[-1],zlim,color,linewidth=2,zorder=1e4)

plt.savefig('./figures/3D_ecco_axes_region2_Ra.png')
ax._axis3don = False
plt.savefig('./figures/3D_ecco_noaxes_region2_Ra.png')